# Merge playlists
#### Probably the easiest way to make an OAuth Token is through this link: https://developer.spotify.com/console/post-playlists/

In [44]:
def getInfo():
    token = input("Spotify OAuth Token: ")
    playlist_name = input("Playlist name: ")
    playlist_description = input("Playlist description: ")
    public = False
    if(input("Should it be public? (1-yes, 0-no) ") == '1'):
        public = True
    return token, playlist_name, playlist_description, public

In [45]:
def createPlaylist(token, playlist_name, playlist_description, public):
    endpoint_url = f"https://api.spotify.com/v1/me/playlists?"
    request_body = json.dumps({
              "name": playlist_name,
              "description": playlist_description,
              "public": public
            })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", "Authorization":f'Bearer {token}'})
    if(response.status_code == 201):
        playlist_id = response.json()['id']
        print(f'Created playlist with id {playlist_id}.')
    else:
        print("Could not create playlist.")
    
    return playlist_id

In [46]:
def findNumOfSongs(token, playlist_id):
    endpoint_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks?'
    query = f'{endpoint_url}'
    response = requests.get(query, headers={"Content-Type":"application/json", "Authorization":f'Bearer {token}'})
    json_response = response.json()
    N = json_response['total']
    print(f'Found {N} Songs.')
    return N

In [47]:
def scrapeNext50(limit, offset, token, playlist_id, all_uris):
    market="ES"
    uris = []
    endpoint_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks?'
    query = f'{endpoint_url}limit={limit}&market={market}&offset={offset}'
    response = requests.get(query, headers={"Content-Type":"application/json", "Authorization":f'Bearer {token}'})
    if(response.status_code != 200):
        print("Could not scrape songs.")
    json_response = response.json()

    for song in json_response['items']:
        if(song['track']['uri'] not in all_uris):
            uris.append(song['track']['uri'])
    
    return uris

In [48]:
def addSongs(uris, my_playlist_id, token):
    
    endpoint_url = f'https://api.spotify.com/v1/playlists/{my_playlist_id}/tracks?'
    
    request_body = json.dumps({
              "uris" : uris
            })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", "Authorization":f'Bearer {token}'})
    
    print(f"Added {len(uris)} songs.")

In [49]:
import requests
import json

token, playlist_name, playlist_description, public = getInfo()
my_playlist_id = createPlaylist(token, playlist_name, playlist_description, public)
playlist_ids = ['76J13PyJx0Od8QGr5L9iVe', '2fndSJN2BPfOoYiQP8OFUT', '7JH2LcqndIxcJwNGphMiNl', '53lUbUbI4qxfEes4fxkiEo']

limit = 50
all_uris = []

for playlist_id in playlist_ids:
    N = findNumOfSongs(token, playlist_id)
    rem = N%limit
    max_loop = N-rem
    for offset in range(0, max_loop, 50):
        uris = scrapeNext50(limit, offset, token, playlist_id, all_uris)
        addSongs(uris, my_playlist_id, token)
    uris = scrapeNext50(rem, max_loop, token, playlist_id, all_uris)
    all_uris += uris
    addSongs(uris, my_playlist_id, token)
    print("Songs added to playlist.")

Spotify OAuth Token: BQBgdEfrjepcuIRpf5yCPOf62mrwYPAr4tnDRRL-BOjoIzXKVo32QvnC5sYPfbrZ4jPIhq-4uXxol4uqiLpOmhyWVIgG-MwRTKMUqo_8o7VUxQQSA4tbXhwSLhdovs05zPy4QPjUfuQjbLAygU-13nFvlzqH_mczkUIZCmXP6ABa3CYwnOWqU1mOCz1zk5QtKhyZWhLaOlmvu_36EY5DltD81dHLT53WYCyWCVS01VjGVc1oX9g-mK_IOmjmaYXFY6Fk5vrwO8zCeX9J5Fo3uNQ
Playlist name: ***
Playlist description: *
Should it be public? (1-yes, 0-no) 1
Created playlist with id 6DzkZbxnwxAVmfyF1W7GbX.
Found 64 Songs.
Added 50 songs.
Added 14 songs.
Songs added to playlist.
Found 160 Songs.
Added 46 songs.
Added 49 songs.
Added 50 songs.
Added 10 songs.
Songs added to playlist.
Found 14 Songs.
Added 11 songs.
Songs added to playlist.
Found 25 Songs.
Added 4 songs.
Songs added to playlist.
